<a href="https://colab.research.google.com/github/YUIMETALFOREVER/KGC2019/blob/master/KGC2019_NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naive Bays Classifierのよる犯人推測**

**セリフデータの読み込み**

　犯人のセリフ　＝＝＞　cri

　犯人以外のセリフ＝＞　inn

 事前作業：
 犯人のセリフデータ'talk_cri.txt'と犯人以外のセリフデータ'talk_inn.txt'をGithubからコピーしてColaboratoryにアップロードしてください。

 現時点では「悪魔の足」のセリフデータのみであるため、学習には大幅に不足しており、pythonコードの動作確認にしかなりません。


In [0]:
import nltk

critalk_file = 'talk_cri.txt'  #セリフデータの指定
inntalk_file = 'talk_inn.txt'

with open(critalk_file,'r') as f:
  critalks = f.readlines()
with open(inntalk_file,'r') as f:
  inntalks = f.readlines()

critalks = [line.lower().strip("\n") for line in critalks]
inntalks = [line.lower().strip("\n") for line in inntalks]

# 文末の記号を直前の単語から分離
critalks = [line.replace("?"," ?").replace("!"," !").replace(".", " .").replace(",", " ,").replace(";"," ;").replace(":", " :") for line in critalks]
inntalks = [line.replace("?"," ?").replace("!"," !").replace(".", " .").replace(",", " ,").replace(";"," ;").replace(":", " :")  for line in inntalks]




セリフデータを単語Listに分離してストップワードを除去



In [99]:
#stopwords を取得
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

def get_vocabulary(talks1, talks2):
  # 単語単位に分割
  split1 = [word for line in talks1 for word in line.split()]
  split2 = [word for line in talks2 for word in line.split()]
  wordslist = [item for list in [split1,split2] for item in list]
  # 重複排除
  wordsset = list(set(wordslist))
  # stopwordsの排除
  vocab =[item for item in wordsset if item not in stopwords_en]
  return vocab

vocabulary = get_vocabulary(critalks, inntalks)
print(vocabulary[0:10])


['poor', 'brothers', 'symptoms', 'money', 'business', 'housekeeper', 'absolutely', 'far', 'dead', "i'll"]


**ラベル付け**

In [101]:
def get_taggeddata(cri, inn):
  # セリフlistを辞書化する　key='talk' セリフを格納　key='label' 'cri' or 'inn' のラベルを格納
  cri_dict = [{'talk':talk.split(),'label':'cri'} for talk in cri]
  inn_dict = [{'talk':talk.split(),'label':'inn'} for talk in inn]
  conv_dict = [item for list in [cri_dict,inn_dict] for item in list]
  print(conv_dict[0:10])
  tagged_data = [(talk['talk'],talk['label']) for talk in conv_dict]
  return tagged_data

tagged_data = get_taggeddata(cri=critalks,inn=inntalks)
print(tagged_data)

[{'talk': ['mr', '.', 'holmes', ',the', 'most', 'extraordinary', 'and', 'tragic', 'affair', 'has', 'occurred', 'during', 'the', 'night', '.', 'it', 'is', 'the', 'most', 'unheard-of', 'business', '.', 'we', 'can', 'only', 'regard', 'it', 'as', 'a', 'special', 'providence', 'that', 'you', 'should', 'chance', 'to', 'be', 'here', 'at', 'the', 'time', ',', 'for', 'in', 'all', 'england', 'you', 'are', 'the', 'one', 'man', 'we', 'need', '.'], 'label': 'cri'}, {'talk': ['perhaps', 'i', 'had', 'best', 'say', 'a', 'few', 'words', 'first', ',and', 'then', 'you', 'can', 'judge', 'if', 'you', 'will', 'listen', 'to', 'the', 'details', 'from', 'mr', '.', 'tregennis', ',', 'or', 'whether', 'we', 'should', 'not', 'hasten', 'at', 'once', 'to', 'the', 'scene', 'of', 'this', 'mysterious', 'affair', '.'], 'label': 'cri'}, {'talk': ['i', 'may', 'explain', ',', 'then', ',', 'that', 'our', 'friend', 'here', 'spent', 'last', 'evening', 'in', 'the', 'company', 'of', 'his', 'two', 'brothers', ','], 'label': 'cri

**学習**

In [112]:
def extract_features(talks):
  talk_words = set(talks)
  features = {}
  for word in vocabulary:
    features[word] = (word in talk_words)
  return features

features = nltk.classify.apply_features(extract_features, tagged_data)

#testデータとtrainデータに分割
split = 65
test_data = features[split+1:]
train_data = features[:split]
print(train_data[0:100])
print(len(test_data),len(train_data))


[({'poor': False, 'brothers': False, 'symptoms': False, 'money': False, 'business': True, 'housekeeper': False, 'absolutely': False, 'far': False, 'dead': False, "i'll": False, 'mile': False, 'frightened': False, 'make': False, 'suggested': False, 'evening': False, 'particular': False, 'live': False, 'judge': False, 'candles': False, 'deny': False, 'taking': False, 'occurs': False, 'power': False, 'fright': False, 'message': False, 'yes': False, 'singing': False, 'world': False, 'tregennis': False, 'spent': False, 'bush': False, 'faces': False, 'call': False, 'could': False, 'dr': False, 'sister': False, 'really': False, 'facing': False, 'explained': False, 'take': False, 'thought': False, 'told': False, 'loose': False, 'doctor': False, 'signs': False, 'spread': False, 'door': False, 'kind': False, 'tredannick': False, 'done': False, 'holmes': True, 'porter': False, 'overtook': False, 'breakfast': False, 'england': True, 'wider': False, 'tragic': True, 'naturally': False, 'mind': False

In [106]:
#学習
trained_nbc = nltk.NaiveBayesClassifier.train(train_data)
print(trained_nbc.show_most_informative_features(10))

#テスト
accuracy = nltk.classify.accuracy(trained_nbc, testData)
print(accuracy)


Most Informative Features
                     two = True              cri : inn    =      3.6 : 1.0
                     sat = True              inn : cri    =      3.1 : 1.0
                       ? = True              inn : cri    =      3.1 : 1.0
                  george = True              inn : cri    =      2.4 : 1.0
                    well = True              inn : cri    =      2.4 : 1.0
                   chair = True              inn : cri    =      2.4 : 1.0
                   could = True              inn : cri    =      2.4 : 1.0
                    back = True              cri : inn    =      2.3 : 1.0
                brothers = True              cri : inn    =      2.3 : 1.0
                     sir = True              inn : cri    =      1.7 : 1.0
None
0.7142857142857143


**分類器の確認**

In [111]:
def nbc_criminal_calc(talk):
  talkwork = talk.lower().strip("\n")
  talkwork = talkwork.replace("?"," ?").replace("!"," !").replace(".", " .").replace(",", " ,").replace(";"," ;").replace(":", " :")
  prob_talk = talkwork.split()
  prob_features = extract_features(prob_talk)
  return trained_nbc.classify(prob_features)

nbc_criminal_calc("Homels, you coward!")

'cri'

最も原始的な仕組みとしては、SPARQLで取得した各登場人物のセリフをコピー＆ペーストして分類器にかけ、登場人物ごとに結果を集計する。